In [1]:
# Melakukan impor libraries yang diperlukan untuk visualisasi data.
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import cv2
import time
import datetime

In [2]:
# Melakukan impor MTCNN untuk deteksi wajah.
! pip install mtcnn
from mtcnn.mtcnn import MTCNN

     |████████████████████████████████| 2.3 MB 3.0 MB/s 


In [3]:
# Mencatat waktu dimulainya keseluruhan program visualisasi data.
global_start_time = time.time()

In [4]:
# Membaca sekaligus menampilkan id dari data testing.
df_test = pd.read_csv('../input/bdc-satriadata2021/submission.csv')
df_test

,id
0,005093b2-8c4b-4ed7-91c3-f5f4d50f8d27
1,0052554e-069e-4c43-beb0-0885e8f7684e
2,0092b954-1143-4a95-a17b-1edfa6af3b01
3,009fc28b-fe9b-441d-b8a2-ea8b7ae6ca16
4,00d0e306-06fe-45d8-ae6c-6f83ab8f7810
...,...
985,feb9e70f-4182-4500-866d-9b95657e727c
986,fed67ed2-620a-4ebf-b61e-16ef3d9b93f7
987,ff14c77e-c3c2-46db-9341-30de4130ef8a
988,ff662f3f-8c76-4d27-a73e-c43c4353f798


In [5]:
# Mencatat waktu dimulainya pembacaan testing images.
start_time = time.time()

# Mencatat index dari setiap testing images.
img_test = {}
index_test = list(df_test.iloc[:,0])
filename_master_test = '../input/bdc-satriadata2021/Testing/'

for index in index_test:
    # Membaca dan menyimpan testing images.
    img_test[index] = {}
    img_index = index + '.jpg'
    filename_test = filename_master_test + img_index
    img_test[index] = mpimg.imread(filename_test)

# Mencatat waktu berakhirnya pembacaan testing images.
end_time = time.time()

# Menampilkan waktu eksekusi dari pembacaan testing images.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

execution time: 0:00:33.769066


In [6]:
# Mendeklarasikan pendeteksi wajah.
detector = MTCNN()


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

In [7]:
# Mencatat waktu dimulainya pencatatan informasi wajah target.
start_time = time.time()

# Menyiapkan daftar informasi penting tiap wajah target.
important_list_master = []

for element in range(1, 1 + len(index_test)):
    # Mendeteksi wajah.
    index = index_test[element - 1]
    my_image = img_test[index].copy()
    face_location = detector.detect_faces(my_image)
    
    # Menyimpan informasi penting wajah target.
    important_list = [element, index]
    if len(face_location):
        for order in range(len(face_location)):
            important_list = [element, index, order]
            main_face_info = face_location[order]
            important_list += main_face_info['box']
            for key_element in main_face_info['keypoints']:
                # Menyimpan koordinat setiap fitur wajah yang penting.
                important_list += main_face_info['keypoints'][key_element]
            important_list += [main_face_info['confidence']]
            important_list_master.append(important_list)
    else:
        # Memberikan nilai dummy -1 untuk gambar wajah yang gagal terdeteksi.
        important_list += [-1 for element in range(16)]
        important_list_master.append(important_list)

# Penyusunan data yang berisi lokasi wajah target.
facial_location_data = pd.DataFrame(important_list_master)
facial_location_data.columns = ['nomor', 'index_gambar', 'order', 'x_coor', 'y_coor', 'width', 'height',
                                'x_lefteye', 'y_lefteye', 'x_righteye', 'y_righteye', 'x_nose', 'y_nose',
                                'x_mouthleft', 'y_mouthleft', 'x_mouthright', 'y_mouthright', 'confidence']

# Menyimpan catatan informasi wajah target dalam format .csv.
facial_location_data.to_csv('test_facial_data.csv', index = False)

# Mencatat waktu berakhirnya pencatatan informasi wajah target.
end_time = time.time()

# Menampilkan waktu eksekusi dari pencatatan informasi wajah target.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

2021-10-27 08:29:50.940420: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


execution time: 0:18:45.913034


In [8]:
# Mencatat waktu berakhirnya keseluruhan program visualisasi data.
global_end_time = time.time()

# Menampilkan waktu eksekusi dari keseluruhan program visualisasi data.
total_execution_time = datetime.timedelta(seconds = global_end_time - global_start_time)
print("total execution time: %s" % (total_execution_time))

total execution time: 0:19:20.281880
